# Cálculo del PERT de un proyecto

## Datos de partida del PERT

In [1]:
import pandas as pd
from decimal import Decimal, getcontext
import qgrid

getcontext().prec = 10  # Precision interna de trabajo
numero_de_decimales = 0
precision = Decimal(10) ** - numero_de_decimales  # Redondeo de valores de entrada


ModuleNotFoundError: No module named 'qgrid'

In [ ]:
datos_enunciado = pd.DataFrame([
# actividad, nodo_inicial, nodo_final, duracion_planificada, recursos, CUR, duracion_real, coste_real, coste_planificado, realizado
  ('A',          1,            2,           5,                   7,      4,       8,            10,          9,            100 ),
  ('B',          1,            3,           5,                   7,      4,       8,            10,          9,            100 ),
  ('C',          2,            5,           5,                   7,      4,       8,            10,          9,            100 ),
  ('D',          4,            7,           5,                   7,      4,       8,            10,          9,            100 ),
  ('E',          3,            6,           5,                   7,      4,       8,            10,          9,            100 ),
  ('F',          7,            8,           5,                   7,      4,       8,            10,          9,            100 ),
  ('G',          7,           12,           5,                   7,      4,       8,            10,          9,            100 ),
  ('H',          8,            9,           5,                   7,      4,       8,            10,          9,            100 ),
  ('I',          6,           10,           5,                   7,      4,       8,            10,          9,            100 ),
  ('J',          5,            9,           5,                   7,      4,       8,            10,          9,            100 ),
  ('K',         10,           11,           5,                   7,      4,       8,            10,          9,            100 ),
  ('L',         10,           11,           5,                   7,      4,       8,            10,          9,            100 ),
  ('M',          9,           12,           5,                   7,      4,       8,            10,          9,            100 ),
  ('N',         11,           12,           5,                   7,      4,       8,            10,          9,            100 ),
  ('f1',         3,            4,           5,                   7,      4,       8,            10,          9,            100 ),
  ('f2',         2,            4,           5,                   7,      4,       8,            10,          9,            100 ),
  ('f3',         6,            7,           5,                   7,      4,       8,            10,          9,            100 ), 
  ('f4',         5,            8,           5,                   7,      4,       8,            10,          9,            100 ),]
    ,columns=['actividad', 'nodo_inicial','nodo_final','duracion_planificada', 'recursos',
              'cur', 'duracion_real', 'coste_real', 'coste_planificado', 'realizado'])

datos_enunciado

In [ ]:
qgrid_widget = qgrid.show_grid(datos_enunciado, show_toolbar=True)
qgrid_widget

In [ ]:
datos_enunciado = qgrid_widget.get_changed_df()
datos_enunciado.set_index('actividad', inplace=True)

## Cálculo usando Python
### Importamos librerías de Python.
En pmule (project management at ULE) está el código que utilizamos para calcular los valores del PERT.

In [ ]:
#from IPython.display import Image
import importlib
import pmule
importlib.reload(pmule)
from pmule import GrafoProyecto, ValorGanado


La clase PertGraph permite visualizar el grafo PERT, el diagrama de Gantt así como realizar el cálculo de tiempos tempranos, tardíos y de las holguras totales.

In [ ]:
proyecto = GrafoProyecto(datos_enunciado.loc[:, ['nodo_inicial', 'nodo_final']])

### Grafo PERT con numeración de nodos

In [ ]:
proyecto.pert(filename='pert_en_blanco.pdf', duraciones=None, orientation='portrait', tiempos=False, ranksep=1, nodesep=0.5)

### Matriz de Zaderenko 

In [ ]:
zaderenko = proyecto.zaderenko(datos_enunciado['duracion_planificada'])
zaderenko

In [ ]:
resultados_pert = proyecto.calcula_pert(duraciones=datos_enunciado['duracion_planificada'])
resultados_pert['tiempos']

In [ ]:
resultados_pert['H_total']

In [ ]:
camino_critico = proyecto.camino_critico(datos_enunciado['duracion_planificada'])
camino_critico

### Grafo PERT con indicación de tiempos obtenidos de acuerdo a la planificación inicial

In [ ]:
filename='pert_resuelto.png'
proyecto.pert(filename=filename,
              duraciones=datos_enunciado['duracion_planificada'],
              orientation='portrait', tiempos=True, ranksep=1, nodesep=0.5)

### Diagrama de Gantt
Con el nombre de la actividad en la casilla

In [ ]:
proyecto.gantt(duraciones=datos_enunciado['duracion_planificada'],
               representar={key: key for key in datos_enunciado.index} )

Con las casillas en blanco

In [ ]:
proyecto.gantt(duraciones=datos_enunciado['duracion_planificada'])

### Uso de recursos a lo largo del tiempo

In [ ]:
gantt_recursos = proyecto.gantt(duraciones=datos_enunciado['duracion_planificada'],
                                representar=datos_enunciado['recursos'], total='fila')
gantt_recursos

In [ ]:
gantt_recursos.data.loc['Total'].plot(kind='bar');

# Método del valor ganado

In [ ]:
proyecto.gantt(datos_enunciado['duracion_planificada'])

In [ ]:
proyecto.gantt(datos_enunciado['duracion_real'])

In [ ]:
vg = ValorGanado(proyecto)

In [ ]:
gantts = vg.calcula_gantts(duraciones_planificadas   = datos_enunciado['duracion_planificada'],
                           duraciones_reales         = datos_enunciado['duracion_real'],
                           costes_planificados       = datos_enunciado['coste_planificado'],
                           costes_reales             = datos_enunciado['coste_real'],
                           porcentaje_de_completacion= datos_enunciado['realizado'])


In [ ]:
gantts['Gantt_PV']

In [ ]:
gantts['Gantt_AC']

In [ ]:
gantts['Gantt_EV']

In [ ]:
acumulados = gantts['acumulados']
acumulados

In [ ]:
acumulados.plot();